In [7]:
import datetime, itertools, sys
import pandas as ps
from subprocess import Popen, PIPE
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from pymystem3 import Mystem

In [2]:
data = ps.read_csv("data/spelled-f.csv", sep=';', header=None,
                   index_col=0,names=['id','title','text','cluster','date','publisher'])

## Предварительная очистка данных

In [3]:
data = data[~data["cluster"].isin(["-", "S", "Standard "])]
print("Число записей в таблице:", len(data))

Число записей в таблице: 32317


In [5]:
day_counts = {}
for _, row in data.iterrows():
    day_counts[row["date"]] = day_counts.get(row["date"], 0) + 1
max_date = max((count, date) for date, count in day_counts.items())[1]
data_1d = data[data["date"] == max_date]
print("Самый плотный день - {}, число записей: {}".format(max_date, len(data_1d)))

Самый плотный день - 2016-01-27, число записей: 19846


## Лемматизация текстов

In [8]:
m = Mystem()
cluster_index = {cluster: i for i, cluster in enumerate(data["cluster"].unique())}
messages = ["".join(m.lemmatize(row["title"] + ". " + row["text"])) for _, row in data_1d.iterrows()]
clusters = [cluster_index[row["cluster"]] for _, row in data_1d.iterrows()]

In [9]:
print(messages[0])
print(len(messages), len(clusters))

суриков: контактный группа обсуждать реализация минский соглашение. контактный группа по урегулирование на юго-восток украина обсуждать путь практический реализация пункт минский договоренность, сообщать посол рф в белоруссия александр суриков. …сейчас...

19846 19846


## Построение матрицы TF*IDF

In [10]:
tfidf_vectorizer = TfidfVectorizer()
tfidf_matrix = tfidf_vectorizer.fit_transform(messages)
print(tfidf_matrix.shape)

(19846, 30114)


## Эксперимент с косинусовой мерой

In [25]:
index_cluster = {index: cluster for cluster, index in cluster_index.items()}
cluster_counts = {}
for cluster in clusters:
    cluster_counts[cluster] = cluster_counts.get(cluster, 0) + 1
max_cluster = max((count, cluster) for cluster, count in cluster_counts.items())[1]
print("Самый большой кластер - '{}' ({})\nЧисленность - {}".format(
        index_cluster[max_cluster], max_cluster, cluster_counts[max_cluster]))

Самый большой кластер - 'Ученые выяснили, как формируются озера на Титане, спутнике Сатурна' (10776)
Численность - 7


In [27]:
print("Все сообщения кластера:\n")
for i, cluster in enumerate(clusters):
    if cluster == max_cluster:
        print("  " + messages[i])

Все сообщения кластера:

  ученый устанавливать причина появление озеро на титан. сделать сравнительный характеристика крупный спутник сатурн с наш планета, ученый удаваться устанавливать причина образование озеро на поверхность титан. …однако в отличие от земной, состоять...

  титан по свой структура очень похожий на земля. британский ученый предполагать, что крупный спутник сатурн – титан по свой структура иметь много сходство с земля. …также ряд исследование показывать, что при взаимодействие с магнитный поле...

  ученый: атмосфера титан очень похожий на земной. ученый из лондонский университетский колледж утверждать, что атмосфера крупный спутник сатурн - титан очень похожий на земной. …обобщать весь получать данные, исследователь обнаруживать на...

  находить причина появление озеро на титан. а вы знать, что на титан, спутник сатурн существовать озеро? долгий время ученый не удаваться раскрывать тайна титан, какой именно образ там появляться озеро.

  на титан обнаруживать иден

In [29]:
target_messages = [i for i, cluster in enumerate(clusters) if cluster == max_cluster]
ind = target_messages[0]
sim_factors = cosine_similarity(tfidf_matrix[ind:ind+1], tfidf_matrix)
for i in target_messages:
    print(sim_factors[0][i])

1.0
0.280886938106
0.371229636823
0.604793443811
0.320538977237
0.432063765871
0.410332287692


In [33]:
th_values = (0.8, 0.6, 0.4, 0.35, 0.3, 0.25, 0.2)
thresholds = {t: 0 for t in th_values}
for sf in sim_factors[0]:
    for th in th_values:
        if sf >= th:
            thresholds[th] += 1
print(sorted(thresholds.items()))

[(0.2, 12), (0.25, 11), (0.3, 8), (0.35, 5), (0.4, 4), (0.6, 2), (0.8, 1)]


In [ ]:
sim_factors_full = cosine_similarity(tfidf_matrix, tfidf_matrix)